In [4]:
import pandas as pd
import numpy as np
from PIL import Image

"""计算四元数转旋转矩阵"""
def q2rot(qMat):
# qMat格式  qw qx qy qz
    w = qMat[0]
    x = qMat[1]
    y = qMat[2]
    z = qMat[3]
#     rotMat = [[1-2*y**2-2*z**2,2*x*y+2*w*z,2*x*z-2*w*y],
#               [2*x*y-2*w*z,1-2*x**2-2*z**2,2*y*z+2*w*x],
#               [2*x*z+2*w*y,2*y*z-2*w*x+1-2*x**2-2*y**2]]
    
    
    rotMat = np.mat([[1-2*y**2-2*z**2,2*x*y-2*w*z,2*x*z+2*w*y],
              [2*x*y+2*w*z,1-2*x**2-2*z**2,2*y*z-2*w*x],
              [2*x*z-2*w*y,2*y*z+2*w*x,1-2*x**2-2*y**2]])
    return rotMat

"""通过四元数，平移矩阵获得其次矩阵"""
def q2homoRot(qMat,qTrans):
# qMat格式  qw qx qy qz
# qTrans格式 x y z
    qw = qMat[0]
    qx = qMat[1]
    qy = qMat[2]
    qz = qMat[3]
    x = qTrans[0]
    y = qTrans[1]
    z = qTrans[2]
    
    
#     rotMat = [[1-2*y**2-2*z**2,2*x*y+2*w*z,2*x*z-2*w*y],
#               [2*x*y-2*w*z,1-2*x**2-2*z**2,2*y*z+2*w*x],
#               [2*x*z+2*w*y,2*y*z-2*w*x+1-2*x**2-2*y**2]]
    
    
    rotMat = np.mat([
              [1-2*qy**2-2*qz**2,  2*qx*qy-2*qw*qz,   2*qx*qz+2*qw*qy,  x],
              [2*qx*qy+2*qw*qz,    1-2*qx**2-2*qz**2, 2*qy*qz-2*qw*qx,  y],
              [2*qx*qz-2*qw*qy,    2*qy*qz+2*qw*qx,   1-2*qx**2-2*qy**2,z],
              [0,                   0,                  0,                 1]
    ])
    return rotMat

"""A朝向B的向量"""
def a2bVec(corA,corB):

    res = []
    for i in range(len(corA)):
        res.append(corB[i]-corA[i])
    return res

def getPngPix(pngPath ,x = 1,y = 1):
    img_src = Image.open(pngPath)
    img_src = img_src.convert('RGBA')
    img_array = img_src.load()
    data = img_array[x,y]
    img_src.close()
    return data

def danwei(vec):
    res = []
    tem = 0
    for i in vec:
        tem += i**2
    tem = tem**0.5
    for i in vec:
        res.append(i/tem)
    return res

def vectorMinor(v1,v2):
    res = []
    for i in range(len(v1)):
        res.append(v1[i]-v2[i])
    return res
def vectorAdd(v1,v2):
    res = []
    for i in range(len(v1)):
        res.append(v1[i]+v2[i])
    return res

def cross(v1,v2):
    res = [v1[1]*v2[2]-v1[2]*v2[1], v1[2]*v2[0]-v1[0]*v2[2], v1[0]*v2[1]-v1[1]*v2[0] ]
    return res

# BASEDIR = "/home/qys/Documents/project/models/weiShengZhi_night"
BASEDIR = "D:/qys/BHU/science/PaperReview/ColmapModels/weiShengZhi_night"
IMGDIR = BASEDIR+"/dense/images/"

# """
# 根据两点 计算直线方程 x y z
# """
# def getLine()

# colmap文档写了是右手坐标系，相机朝向是z的方向.

In [56]:
# 把训练集弄出来
# 对于每一个uv  看哪些照片中能看到他 对于简单的图像可以看法向 如果钝角 那就看不见
# 知道相机位姿 然后顶点和groundtruth连线就是方向


df = pd.read_table(BASEDIR+"/sparse/images.txt")
# print(df.index)
# print(df.loc[3].values[0].split(" "))

# imagesInfo 每一行是每一个图片 格式为：  IMAGE_ID, QW, QX, QY, QZ, TX, TY, TZ, CAMERA_ID, NAME
imagesInfo = []
vMatrix = []
vnMatrix = []
vtMatrix = []
f = 2887.91
cx = 1368
cy = 1824
WIDTH = 2736
HEIGHT = 3648
cameraMatrix = [[f, 0, cx],
                [0, f, cy],
                [0, 0, 1]]

# 记录每张图片的相机位姿
imgMatrix = []
imgRotMatrix = []
# fMatrix:  三维数组  最里面的是 v  vt  vn 索引
fMatrix = []
for i in range(189):
    infoInt = []
    infotem = df.loc[i*2+3].values[0].split(" ");
    infoInt.append(int(infotem[0]))
    
    for j in range(len(infotem)-2):
        infoInt.append(float(infotem[j+1]))
    infoInt.append(infotem[-1])
    imagesInfo.append(infoInt)
#     print(infoInt)
#     print(infoInt[0])
# print(imagesInfo)
# 对imagesInfo 按序号排序
for i in range(189):
    for j in range(188):
        if imagesInfo[j][0]>imagesInfo[j+1][0]:
            tem = imagesInfo[j+1];
            imagesInfo[j+1] =  imagesInfo[j]
            imagesInfo[j] = tem
# 计算增广矩阵
for i in range(len(imagesInfo)):
    imgRotMatrix.append(q2homoRot(imagesInfo[i][1:5],imagesInfo[i][5:8]))


    
# df1 = pd.read_table(BASEDIR+"/dense/openMVS_scene/dense_texture_cut_changeVN.obj")
# df1 = pd.read_table(BASEDIR+"/dense/openMVS_scene/dense_texture_v1_cut.obj")
df1 = pd.read_table(BASEDIR+"/dense/openMVS_scene/dense_texture_cut.obj")

for i in df1.index:
    lineInfo = df1.loc[i].values[0]
    if(lineInfo[0:2] == "f "):
        fInfo = lineInfo.split(" ")[1:4]
        tem = []
        for j in fInfo:
            a = j.split("/")
            tem2 = []
            for k in a:
                tem2.append(int(k))
            tem.append(tem2)
        fMatrix.append(tem)        
    elif(lineInfo[0:2] == "vt"):
        vtTem = lineInfo.split(" ")[1:3]
        vtIntTem = []
        for j in vtTem:
            vtIntTem.append(float(j))
        vtMatrix.append(vtIntTem)
    elif(lineInfo[0:2] == "vn"):
        vnTem = lineInfo.split(" ")[1:4]
        vnIntTem = []
        for j in vnTem:
            vnIntTem.append(float(j))        
        vnMatrix.append(vnIntTem)
    elif(lineInfo[0:2] == "v "):
        vTem = lineInfo.split(" ")[1:4]
        vIntTem = []
        for j in vTem:
            vIntTem.append(float(j))
        vIntTem.append([])
        vMatrix.append(vIntTem)
        

"""
计算所有图片的位置
"""
for i in imagesInfo:
    rota = q2rot(i[1:5])
    #旋转矩阵的逆 即他的转置
    rotaT = rota.T
    t = i[5:8]
    imgMatrix.append(np.multiply(-1,np.dot(rotaT,t)))
temImgMa = []
for i in imgMatrix:
    testSma = []
    testSma.append(i[0,0])
    testSma.append(i[0,1])
    testSma.append(i[0,2])
    temImgMa.append(testSma)
imgMatrix = temImgMa




In [57]:
# 计算每个点，如果相机朝向的反方向和法向是钝角则这个图片中看不到这个点
# 对于每个顶点，遍历他到所有图片的相机朝向的反方向
# 首先计算所有图片的方向的反方向
# for i in imagesInfo:
#     #方向向量为(0,0,1) 然后需要是反方向

"""
zDirReverse = [0,0,-1]
# w = np.array([[1, 2]])
# x = np.array([[1,2,3],[2,3,4]])
# print(np.dot(w,x))

# 先计算每个图片的相机朝向坐标

q = imagesInfo[0][1:5]
rot = q2rot(q)
matrix = np.dot(rot,zDirReverse)


for i in range(len(imagesInfo)):
    q = imagesInfo[i][1:5]
    rot = q2rot(q)
    matrix = np.dot(rot,zDirReverse)
    imagesInfo[i].append(matrix)
"""


# //可能有多个uv映射到同一个顶点
# 对于每个定点 遍历fMatrix 进行点乘
#  每个点的法向 · 顶点指向相机的向量  如果为负 就是看不见

# 存放对于每一个vt 记录他映射的顶点， 二维  vt编号（从0开始）， v编号（从0开始）
vt_vMatirx = []
for i in range(len(vtMatrix)):
    vt_vMatirx.append([i,0])
# print(len(vt_vMatirx))

# print(fMatrix)
"""
千万注意 f是从1开始编号的
"""
for face in fMatrix:
    for face_in in face:

        vt_vMatirx[face_in[1]-1][1] = face_in[0]-1
print("完结")

# vMatrix = []

# print(imagesInfo)
# 对应列表
for i in range(len(vMatrix)):
    #计算点到相机原点的向量 点乘顶点法向
    vMatrix[i][3] = []
    for img in range(len(imgMatrix)):
        vec = a2bVec(vMatrix[i][0:3],imgMatrix[img])
        mul = np.dot(vnMatrix[i][0:3],vec)
        if(mul>0):
#             print(imagesInfo[img][0])
            vMatrix[i][3].append(imagesInfo[img][0]-1)
            

# print(vMatrix)
# for i in range(len(vMatrix)):
#     vMatrix[i][3] = []
#     for img in imagesInfo:
        
        
#         mul = np.dot(vnMatrix[i][0:3],a2bVec(vMatrix[i][0:3],img[5:8]))
#         if mul > 0:
#             vMatrix[i][3].append(img[9])

#     print(vMatrix[i])




完结


In [33]:
# for t in vt_vMatirx:
#     #对于每个纹理坐标
#     v = vMatrix[t[1]]
#     for img in v[3]:
#         #计算顶点在相机坐标系下的坐标
#         vPos = v[0:3].append(1)
#         PC = np.dot(imgRotMatrix[img],vPos)

v = vMatrix[1]
vPos = v[0:3]
vPos.append(1)
testMat = imgRotMatrix[1]
print(v)
print(vPos)
print(testMat)

test = np.dot(testMat,vPos)
print(test)
print(test[0,0:3])
fuck = test[0,0:3]

test = np.dot(cameraMatrix,fuck.T)
uvPos = test.T.getA()
print(uvPos[0])
# uvPos.getA()
print(uvPos[0][0:2])
# s = np.dot(cameraMatrix,fuck.T)
# tem = s.getA()
# tem[0]

# print(s.getA())

[-1.046294, 2.127272, 1.314371, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188]]
[-1.046294, 2.127272, 1.314371, 1]
[[ 0.99513738  0.06857642 -0.07070267  0.587578  ]
 [-0.06783039  0.99761344  0.01290197 -2.9716

In [100]:
# print(vt_vMatirx)

"""
思路是这样的：对于每一个uv 对应一个顶点，每个顶点到n个图像上对应点有一条直线，直线与图像相交于rgb
利用uv 定位顶点。对于顶点，遍历他的可视图像列表，如果他在这个图片里面（像素小于最大像素）计算这个顶点到图片原点的向量 
（或许可以用map存储）并且新建一条数据

有增广矩阵了 右乘世界坐标系坐标
"""
data_mat = []
mod = 1000

"""读取所有图片 并将它存在数组中"""
imageMatrixDetailed = []
for i in imagesInfo:
    image = Image.open(IMGDIR+i[9])
    imageMatrixDetailed.append(image)
    
with open(BASEDIR+"/ground_truth.txt","w") as f:

#     for i in vMatrix:
#         if i[3] == []:
#             f.write("\nv " + str(i[0]) + " " + str(i[1]) + " " + str(i[2]))
            
    for t in vt_vMatirx:
    
        #对于每个纹理坐标
        if t[0] % 500 ==0:
            print(t[0])
        v = vMatrix[t[1]]
        for img in v[3]:
            #计算顶点在相机坐标系下的坐标
            vPos = v[0:3]
            #转为齐次坐标
            vPos.append(1)
            PC = np.dot(imgRotMatrix[img],vPos)
            PC = PC[0,0:3]
            Pimg = np.dot(cameraMatrix,PC.T)
            Pimg[0] = Pimg[0]/Pimg[2]
            Pimg[1] = Pimg[1]/Pimg[2]
            Pimg[2] = Pimg[2]/Pimg[2]
            uvPos = (Pimg.T).getA()
            uvPos = uvPos[0][0:2]
    #         print(uvPos)
            if(uvPos[0]<WIDTH and uvPos[1]<HEIGHT):
                #计算顶点到图片原点的向量 即x y z
                v2centerVec = a2bVec(v[0:3],imgMatrix[img])
                #计算对应点像素坐标
                image = imageMatrixDetailed[img]
                rgb = image.getpixel((uvPos[0],uvPos[1]))
                rowData = []
                rowData.append(vtMatrix[t[0]])
                rowData.append(v2centerVec)
                rowData.append(rgb)
    #             print(rowData)
                data_mat.append(rowData)

                if(len(data_mat)%mod == 0):
                    #每隔mod个写一次
#                     print("书写第"+str(len(data_mat)/mod))
                    for ind in range(len(data_mat)-mod,len(data_mat)):
                        f.write(str(data_mat[ind][0][0])+" "+str(data_mat[ind][0][1])+" "
                                +str(data_mat[ind][1][0])+" "+str(data_mat[ind][1][1])+" "+str(data_mat[ind][1][2])+" "
                               +str(data_mat[ind][2][0])+" "+str(data_mat[ind][2][1])+" "+str(data_mat[ind][2][2])+"\n")

    #再把多余的写进去
    for ind in range(len(data_mat)-len(data_mat)%mod,len(data_mat)):
        f.write("\n"+str(data_mat[ind][0][0])+" "+str(data_mat[ind][0][1])+" "
                    +str(data_mat[ind][1][0])+" "+str(data_mat[ind][1][1])+" "+str(data_mat[ind][1][2])+" "
                    +str(data_mat[ind][2][0])+" "+str(data_mat[ind][2][1])+" "+str(data_mat[ind][2][2]))
    
                            
                    
                            
            
# print(vMatrix[0:10])
# for t in vt_vMatirx[0:10]:
#     #对于每个纹理坐标
#     v = vMatrix[t[1]]
#     for img in v[3]:`
#         #计算顶点在相机坐标系下的坐标
#         vPos = v[0:3]
#         #转为齐次坐标
#         vPos.append(1)
#         PC = np.dot(imgRotMatrix[img],vPos)
#         PC = PC[0,0:3]
#         Pimg = np.dot(cameraMatrix,PC.T)
#         Pimg[0] = Pimg[0]/Pimg[2]
#         Pimg[1] = Pimg[1]/Pimg[2]
#         Pimg[2] = Pimg[2]/Pimg[2]
#         uvPos = (Pimg.T).getA()
#         uvPos = uvPos[0][0:2]
# #         print(uvPos)
#         if(uvPos[0]<WIDTH and uvPos[1]<HEIGHT):
#             #计算顶点到图片原点的向量 即x y z
#             v2centerVec = a2bVec(v[0:3],imgMatrix[img])
#             #计算对应点像素坐标
#             image = Image.open(IMGDIR+imagesInfo[img][9])
#             rgb = image.getpixel((uvPos[0],uvPos[1]))
#             rowData = []
#             rowData.append(uvPos)
#             rowData.append(v2centerVec)
#             rowData.append(rgb)
# #             print(rowData)
#             data_mat.append(rowData)
#         if(len(data_mat)%100 == 0):
#             #一百条写一次
            
#         test = np.dot(cameraMatrix,fuck.T)   
        
print("结束")

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
结束


In [1]:
print(len(data_mat))

NameError: name 'data_mat' is not defined

In [14]:
# count = 0;
# for i in vMatrix:
#     if i[3] == []:
# #         print()
#         count = count+1;
# print(count)

# for i in range(len(vMatrix)):
#     if vMatrix[i][3] == []:
#         print(i,vnMatrix[i][0],vnMatrix[i][1],vnMatrix[i][2])

# with open(BASEDIR+"/test.obj","w") as f:
# #     f.write("这是个测试！")  # 自带文件关闭功能，不需要再写f.close()
#     for i in vMatrix:
#         if i[3] == []:
#             f.write("\nv " + str(i[0]) + " " + str(i[1]) + " " + str(i[2]))


0


In [34]:
temImg = imagesInfo[0]
imgdir = temImg[9]
temImg
imgdir
img = Image.open(IMGDIR+imgdir)
print("图片大小")
print(img.size)
print(img.format)
print(img.mode)
img.show()
# img.thumbnail(5)
rgb = img.getpixel((126,52))
print(img.getpixel((126,52)))

图片大小
(2736, 3648)
JPEG
RGB
(17, 13, 14)


In [52]:
# print(data_mat[0:5])
"""读取文件数据"""
df = pd.read_table(BASEDIR+"/ground_truth.txt")

print(len(df))
print(df.loc[0].values[0])
print(df.loc[1].values[0])



# # 记录每张图片的相机位姿
# imgMatrix = []
# imgRotMatrix = []
# # fMatrix:  三维数组  最里面的是 v  vt  vn 索引
# fMatrix = []
# for i in range(189):
#     infoInt = []
#     infotem = df.loc[i*2+3].values[0].split(" ");
# data

2420787
0.007763 0.876986 0.13668124623421962 0.860689078087141 -3.7103772403323756 114 82 25
0.007763 0.876986 0.3669647688188862 0.8573756542490631 -3.739361696833522 114 81 27


In [82]:
data_mat = []
for i in range(len(df)):
# for i in range(5):
    rowData = []
    tem = df.loc[i].values[0].split(" ")
#     print(tem)
#     uv = []
#     xyz = []
#     rgb = []
    for j in range(8):
        rowData.append(float(tem[j]))
#     for j in range(2,5):
#         rowData.append(float(tem[j]))
#     for j in range(5,8):
#         rowData.append(int(tem[j]))
   
    data_mat.append(rowData)
print(len(data_mat))

data_mat = torch.from_numpy(np.array(data_mat))
data_mat

2420787


tensor([[ 7.7630e-03,  8.7699e-01,  1.3668e-01,  ...,  1.1400e+02,
          8.2000e+01,  2.5000e+01],
        [ 7.7630e-03,  8.7699e-01,  3.6696e-01,  ...,  1.1400e+02,
          8.1000e+01,  2.7000e+01],
        [ 7.7630e-03,  8.7699e-01,  4.5336e-01,  ...,  1.1300e+02,
          8.2000e+01,  2.5000e+01],
        ...,
        [ 6.6724e-01,  9.1162e-01, -3.2122e+00,  ...,  1.1300e+02,
          7.9000e+01,  3.3000e+01],
        [ 6.6724e-01,  9.1162e-01, -3.3438e+00,  ...,  1.1900e+02,
          8.3000e+01,  3.5000e+01],
        [ 6.6724e-01,  9.1162e-01, -3.3153e+00,  ...,  1.1500e+02,
          8.1000e+01,  3.5000e+01]], dtype=torch.float64)

In [122]:
"""在这里进行神经网络的定义"""
import torch.nn as nn
import torch.nn.functional as F
import torch

# class NetL1(nn.Module):
#     def __init__(self):
#         super(NetL1, self).__init__()
#         self.fc1 = nn.Sequential(
#             nn.Linear(2, 512),
#             nn.ReLU()
#         )

        

#     def forward(self, uvMatrix):
#         """
#         传uv坐标
#         """
#         fc1_out = self.fc1(uvMatrix)

#         # fc6_out = self.fc6(torch.cat((x, fc5_out), 0))
#         # sigma = self.addiSig(fc8_out)

#         out = self.fc2(fc1_out)

#         return out

#     def num_flat_features(self, x):
#         size = x.size()[1:]  # all dimensions except the batch dimension
#         num_features = 1
#         for s in size:
#             num_features *= s
#         return num_features


# class NetL2(nn.Module):
#     def __init__(self):
#         super(NetL2, self).__init__()
#         self.fc1 = nn.Sequential(
#             nn.Linear(3, 512),
#             nn.ReLU()
#         )

#         self.fc2 = nn.Sequential(
#             nn.Linear(512, 256),
#             nn.Sigmoid()
#         )

#         self.fc3 = nn.Sequential(
#             nn.Linear(256, 192),
#             nn.Sigmoid()
#         )

#     def forward(self, posMatrix):
#         """
#         前馈神经网络
#         :param x:位置，向量
#         :param dire: 方向
   
#         """
#         fc1_out = self.fc1(posMatrix)
#         fc2_out = self.fc2(fc1_out)
#         out = self.fc3(fc2_out)

#         return out

class NetFC(nn.Module):
    def __init__(self):
        super(NetFC, self).__init__()
        
        """左上角uv模块"""
        self.uvfc1 = nn.Sequential(
            nn.Linear(2, 512),
            nn.ReLU()
        )
        
        self.uvfc2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.Sigmoid()
        )
        
        """左下角xyz模块"""
        self.xyzfc1 = nn.Sequential(
            nn.Linear(3, 512),
            nn.ReLU()
        )

        self.xyzfc2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.Sigmoid()
        )

        self.xyzfc3 = nn.Sequential(
            nn.Linear(256, 192),
            nn.Sigmoid()
        )
        
        
        """总模块"""
        self.fc1 = nn.Sequential(
            nn.Linear(448, 1000),
            nn.ReLU()
        )

        self.fc2 = nn.Sequential(
            nn.Linear(1000, 800),
            nn.Sigmoid()
        )

        self.fc3 = nn.Sequential(
            nn.Linear(800, 600),
            nn.Sigmoid()
        )

        #加上跳跃层 进行拼接  256 + 600
        self.fc4 = nn.Sequential(
            nn.Linear(856, 3),
            nn.Sigmoid()
        )

    def forward(self, uv,xyz):
        """
        :param uv:
        :param xyz:
        :return:
        """
        
        uvfc1_out = self.uvfc1(uv.to(torch.float32))
        uv_out = self.uvfc2(uvfc1_out)
        
        xyzfc1_out = self.xyzfc1(xyz.to(torch.float32))
        xyzfc2_out = self.xyzfc2(xyzfc1_out)
        xyzfc_out = self.xyzfc3(xyzfc2_out)
        
        
        inputMatrix = torch.cat((uv_out,xyzfc_out),-1)
        fc1_out = self.fc1(inputMatrix)
        fc2_out = self.fc2(fc1_out)
        fc3_out = self.fc3(fc2_out)
        out = self.fc4(torch.cat((uv_out,fc3_out),-1))

        return out

In [90]:
a = data_mat[1:3,1:3]
b = data_mat[3:5,1:3]
# # b[2][1]  = 1
test = torch.cat((a,b),-1)
print(a)
print(b)
test

tensor([[0.8770, 0.3670],
        [0.8770, 0.4534]], dtype=torch.float64)
tensor([[0.8770, 0.5425],
        [0.8770, 0.6355]], dtype=torch.float64)


tensor([[0.8770, 0.3670, 0.8770, 0.5425],
        [0.8770, 0.4534, 0.8770, 0.6355]], dtype=torch.float64)

In [51]:

y = [[1,2,3], [2,3,4], [1,3,4]]
# variabley = torch.Tensor(y) 
# print(y)  
# print(variabley)
# print(data_mat[0])
# uv = torch.Tensor(data_mat[0][1])
fuck = np.array(y)
print(fuck)
uv = torch.from_numpy(fuck)
print(uv)
# uv[0:2]

[[1 2 3]
 [2 3 4]
 [1 3 4]]
tensor([[1, 2, 3],
        [2, 3, 4],
        [1, 3, 4]], dtype=torch.int32)


In [125]:
# print(data_mat[0][0])
print(len(data_mat))

2420787


In [ ]:
"""
网络训练
"""
net = NetFC()

loss_function = nn.KLDivLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.1, momentum=0.9)


for epoch in range(20):
    print("epoch:"+str(epoch))
    for i in range(len(data_mat)):
        RGB = net.forward(data_mat[i][0:2],data_mat[i][2:5])
    #     print(RGB)
        loss = loss_function(RGB, data_mat[i][5:8].to(torch.float32))

        if(i%10000==0):
            print(str(i)+str(loss))

        loss.backward()
        optimizer.step()
    

# for i in range(500)
#     outUV = 

# print(outRGB)


epoch:0
0tensor(285.3492, grad_fn=<KlDivBackward>)
10000tensor(414.4618, grad_fn=<KlDivBackward>)
20000tensor(309.3951, grad_fn=<KlDivBackward>)
30000tensor(155.6033, grad_fn=<KlDivBackward>)
40000tensor(315.7475, grad_fn=<KlDivBackward>)
50000tensor(358.7525, grad_fn=<KlDivBackward>)
60000tensor(645.2883, grad_fn=<KlDivBackward>)
70000tensor(895.1122, grad_fn=<KlDivBackward>)
80000tensor(680.9998, grad_fn=<KlDivBackward>)
90000tensor(964.8040, grad_fn=<KlDivBackward>)
100000tensor(812.1843, grad_fn=<KlDivBackward>)


In [11]:
# for i in range(len(fMatrix)):
#     for j in fMatrix[i]:
#         if j[0] == 16:
#            print(fVn[i])
# #            tem = []
# #            for k in range(3):
# #             tem.append(fMatrix[i][k][0]-1)
# #            fuck.append(tem)
# #             pointVn = vectorAdd(pointVn,fVn[i])

# a = [-0.11916593,0.41345999,-0.9026906 ]
# point = [-1.615437, 2.821256, 1.006795]

# def danwei(vec):
#     res = []
#     tem = 0
#     for i in vec:
#         tem += i**2
#     tem = tem**0.5
#     for i in vec:
#         res.append(i/tem)
#     return res
# # 对应列表
# # for img in imgMatrix:
# #     vec = a2bVec(point,i)
# #     mul = np.dot(a,vec)
# #     print(img[5:8])
# #     print(danwei(vec))
# #     print(mul)
# #     print("------")
# #     if(mul>0):
# #         print("******!!!!!!")
# #         print(mul)


# with open(BASEDIR+"/images.obj","w") as f:
# #     f.write("这是个测试！")  # 自带文件关闭功能，不需要再写f.close()
#     for i in imgMatrix:
        
#         f.write("\nv " + str(i[0]) + " " + str(i[1]) + " " + str(i[2]))

# # print(vMatrix)

[-0.14407773  0.36052423 -0.92155515]
[-0.11916593  0.41345999 -0.9026906 ]
[-0.08857717  0.41036694 -0.90760843]
[-0.14045937  0.34977787 -0.92624328]
[-0.12468025  0.36493511 -0.92264685]


In [24]:
# fVn 是存放有自己计算的vn
fVn = np.zeros((len(fMatrix),3))
for f in range(len(fMatrix)):
    v1 = vectorMinor(vMatrix[fMatrix[f][1][0]-1][0:3],vMatrix[fMatrix[f][0][0]-1][0:3])
    v2 = vectorMinor(vMatrix[fMatrix[f][2][0]-1][0:3],vMatrix[fMatrix[f][0][0]-1][0:3])
    temRes = cross(v1,v2);
    for i in range(3):
        fVn[f][i] = temRes[i]

# 对VN单位化
for i in fVn:
    mod = (i[0]**2+i[1]**2+i[2]**2)**0.5
    for j in range(len(i)):
        i[j] = i[j]/mod
# print(fVn)


fuck = []
pointSetVn = np.zeros((len(vMatrix),3))
for i in range(len(fMatrix)):
    for j in fMatrix[i]:
        pointSetVn[j[0]-1] = vectorAdd(pointSetVn[j[0]-1],fVn[i])
        
# 单位化
for i in pointSetVn:
    mod  = (i[0]**2+i[1]**2+i[2]**2)**0.5
#     print("**")
#     print(mod)
    for j in range(len(i)):
        i[j] = i[j]/mod
#         print("j:" + str(j))
    
pointVn = [0,0,0]
print(vnMatrix[0:16])
print(pointSetVn[0:16])

[[0.016732, -0.789287, -0.613796], [0.026535, -0.79249, -0.609307], [0.022581, -0.772494, -0.634621], [-0.042018, -0.744069, -0.666781], [-0.054113, -0.742447, -0.667715], [-0.014808, -0.765256, -0.643555], [0.116283, -0.6704, 0.732832], [0.148128, -0.660517, 0.736054], [0.135497, -0.632305, 0.762779], [0.27155, -0.954924, -0.119917], [0.25633, -0.966563, -0.007147], [0.183768, -0.933266, 0.308617], [0.006676, -0.755387, -0.655245], [-0.004575, -0.757903, -0.652351], [0.024184, -0.7658, -0.642624], [-0.124468, 0.38096, -0.916176]]
[[ 1.56973841e-02 -7.90604961e-01 -6.12125304e-01]
 [ 2.99901697e-02 -7.93385699e-01 -6.07980034e-01]
 [ 1.90080612e-02 -7.72595824e-01 -6.34613572e-01]
 [-3.72415748e-02 -7.46223537e-01 -6.64652916e-01]
 [-5.37461502e-02 -7.42508070e-01 -6.67677405e-01]
 [-1.46573917e-02 -7.66793828e-01 -6.41726099e-01]
 [ 1.16297960e-01 -6.71622442e-01  7.31709013e-01]
 [ 1.48238244e-01 -6.60566966e-01  7.35986893e-01]
 [ 1.36720300e-01 -6.32055494e-01  7.62766944e-01]
 [ 2

<ipython-input-24-ac9b51e83116>:49: RuntimeWarning: invalid value encountered in double_scalars
  i[j] = i[j]/mod
